# Cleaning data

## Setup

In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

FOLDERNAME = "AWID"

%cd /content/drive/My\ Drive/$FOLDERNAME

Mounted at /content/drive
/content/drive/My Drive/AWID


In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from IPython.display import display
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
import missingno as msno

In [ ]:
# Display all the dataFrame
def display_all(df):
    with pd.option_context('display.max_rows', 1000, 'display.max_columns', 1000):
        display(df)

In [ ]:
train_path = './preprocessedData/train.csv'
test_path = './preprocessedData/test.csv'

In [ ]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [ ]:
train_df

,frame.time_delta,frame.time_delta_displayed,frame.len,frame.cap_len,radiotap.length,radiotap.present.tsft,radiotap.present.flags,radiotap.present.channel,radiotap.present.dbm_antsignal,radiotap.present.antenna,radiotap.present.rxflags,radiotap.flags.cfp,radiotap.flags.preamble,radiotap.flags.wep,radiotap.flags.frag,radiotap.flags.fcs,radiotap.flags.datapad,radiotap.flags.badfcs,radiotap.flags.shortgi,radiotap.datarate,radiotap.channel.freq,radiotap.channel.type.turbo,radiotap.channel.type.cck,radiotap.channel.type.ofdm,radiotap.channel.type.2ghz,radiotap.channel.type.5ghz,radiotap.channel.type.passive,radiotap.channel.type.dynamic,radiotap.channel.type.gfsk,radiotap.channel.type.gsm,radiotap.channel.type.sturbo,radiotap.channel.type.half,radiotap.channel.type.quarter,radiotap.dbm_antsignal,radiotap.antenna,radiotap.rxflags.badplcp,wlan.fc.type_subtype,wlan.fc.type,wlan.fc.subtype,wlan.fc.ds,wlan.fc.frag,wlan.fc.retry,wlan.fc.pwrmgt,wlan.fc.moredata,wlan.fc.protected,wlan.duration,wlan.ra,wlan.da,wlan.ta,wlan.sa,wlan.bssid,wlan.frag,wlan.seq,wlan.fcs_good,class
0,0.000000,0.000000,261,261,26,1,1,1,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2437.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-47.0,1.0,0.0,0x08,0,8,0x00,0,0,0,0,0,0.0,ff:ff:ff:ff:ff:ff,ff:ff:ff:ff:ff:ff,b0:48:7a:e2:62:23,b0:48:7a:e2:62:23,b0:48:7a:e2:62:23,0.0,2851.0,1.0,normal
1,0.024271,0.024271,185,185,26,1,1,1,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2437.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-47.0,1.0,0.0,0x08,0,8,0x00,0,0,0,0,0,0.0,ff:ff:ff:ff:ff:ff,ff:ff:ff:ff:ff:ff,38:22:9d:ae:c5:65,38:22:9d:ae:c5:65,38:22:9d:ae:c5:65,0.0,2159.0,1.0,normal
2,0.001631,0.001631,185,185,26,1,1,1,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2437.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-64.0,1.0,0.0,0x08,0,8,0x00,0,0,0,0,0,0.0,ff:ff:ff:ff:ff:ff,ff:ff:ff:ff:ff:ff,38:22:9d:c4:8d:28,38:22:9d:c4:8d:28,38:22:9d:c4:8d:28,0.0,291.0,1.0,normal
3,0.055325,0.055325,159,159,26,1,1,1,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2437.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-32.0,1.0,0.0,0x08,0,8,0x00,0,0,0,0,0,0.0,ff:ff:ff:ff:ff:ff,ff:ff:ff:ff:ff:ff,00:13:33:87:62:6d,00:13:33:87:62:6d,00:13:33:87:62:6d,0.0,3684.0,1.0,normal
4,0.000415,0.000415,54,54,26,1,1,1,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,48.0,2437.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-21.0,1.0,0.0,0x24,2,4,0x01,0,0,0,0,0,44.0,28:c6:8e:86:d3:d6,28:c6:8e:86:d3:d6,00:25:bc:ed:07:cf,00:25:bc:ed:07:cf,28:c6:8e:86:d3:d6,0.0,3874.0,1.0,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795570,0.000006,0.000006,40,40,26,1,1,1,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,24.0,2437.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-25.0,1.0,0.0,0x1d,1,13,0x00,0,0,0,0,0,0.0,c0:18:85:94:b6:55,NaN,NaN,NaN,NaN,NaN,NaN,1.0,normal
1795571,0.005067,0.005067,148,148,26,1,1,1,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2437.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-58.0,1.0,0.0,0x08,0,8,0x00,0,0,0,0,0,0.0,ff:ff:ff:ff:ff:ff,ff:ff:ff:ff:ff:ff,28:c6:8e:86:d3:d6,28:c6:8e:86:d3:d6,28:c6:8e:86:d3:d6,0.0,2134.0,1.0,normal
1795572,0.010242,0.010242,54,54,26,1,1,1,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,54.0,2437.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-27.0,1.0,0.0,0x24,2,4,0x01,0,0,0,0,0,44.0,28:c6:8e:86:d3:d6,28:c6:8e:86:d3:d6,20:64:32:54:c9:da,20:64:32:54:c9:da,28:c6:8e:86:d3:d6,0.0,1442.0,1.0,normal
1795573,0.000007,0.000007,40,40,26,1,1,1,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,24.0,2437.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-25.0,1.0,0.0,0x1d,1,13,0x00,0,0,0,0,0,0.0,20:64:32:54:c9:da,NaN,NaN,NaN,NaN,NaN,NaN,1.0,normal


In [ ]:
test_df

,frame.time_delta,frame.time_delta_displayed,frame.len,frame.cap_len,radiotap.length,radiotap.present.tsft,radiotap.present.flags,radiotap.present.channel,radiotap.present.dbm_antsignal,radiotap.present.antenna,radiotap.present.rxflags,radiotap.flags.cfp,radiotap.flags.preamble,radiotap.flags.wep,radiotap.flags.frag,radiotap.flags.fcs,radiotap.flags.datapad,radiotap.flags.badfcs,radiotap.flags.shortgi,radiotap.datarate,radiotap.channel.freq,radiotap.channel.type.turbo,radiotap.channel.type.cck,radiotap.channel.type.ofdm,radiotap.channel.type.2ghz,radiotap.channel.type.5ghz,radiotap.channel.type.passive,radiotap.channel.type.dynamic,radiotap.channel.type.gfsk,radiotap.channel.type.gsm,radiotap.channel.type.sturbo,radiotap.channel.type.half,radiotap.channel.type.quarter,radiotap.dbm_antsignal,radiotap.antenna,radiotap.rxflags.badplcp,wlan.fc.type_subtype,wlan.fc.type,wlan.fc.subtype,wlan.fc.ds,wlan.fc.frag,wlan.fc.retry,wlan.fc.pwrmgt,wlan.fc.moredata,wlan.fc.protected,wlan.duration,wlan.ra,wlan.da,wlan.ta,wlan.sa,wlan.bssid,wlan.frag,wlan.seq,wlan.fcs_good,class
0,0.000000,0.000000,1552,1552,26,1,1,1,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,54.0,2437.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-25.0,1.0,0.0,0x28,2,8,0x02,0,0,0,0,1,44.0,c0:18:85:94:b6:55,c0:18:85:94:b6:55,28:c6:8e:86:d3:d6,00:13:33:87:62:6d,28:c6:8e:86:d3:d6,0.0,2313.0,1.0,normal
1,0.001056,0.001056,1552,1552,26,1,1,1,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,54.0,2437.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-25.0,1.0,0.0,0x28,2,8,0x02,0,1,0,0,1,44.0,c0:18:85:94:b6:55,c0:18:85:94:b6:55,28:c6:8e:86:d3:d6,00:13:33:87:62:6d,28:c6:8e:86:d3:d6,0.0,2314.0,1.0,normal
2,0.000141,0.000141,40,40,26,1,1,1,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,24.0,2437.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-26.0,1.0,0.0,0x1d,1,13,0x00,0,0,0,0,0,0.0,c0:18:85:94:b6:55,NaN,NaN,NaN,NaN,NaN,NaN,1.0,normal
3,0.000156,0.000156,40,40,26,1,1,1,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,24.0,2437.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-26.0,1.0,0.0,0x1d,1,13,0x00,0,0,0,0,0,0.0,c0:18:85:94:b6:55,NaN,NaN,NaN,NaN,NaN,NaN,1.0,normal
4,0.001842,0.001842,1552,1552,26,1,1,1,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,54.0,2437.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-26.0,1.0,0.0,0x28,2,8,0x02,0,0,0,0,1,44.0,c0:18:85:94:b6:55,c0:18:85:94:b6:55,28:c6:8e:86:d3:d6,00:13:33:87:62:6d,28:c6:8e:86:d3:d6,0.0,2318.0,1.0,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575638,0.000643,0.000643,40,40,26,1,1,1,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,24.0,2437.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-25.0,1.0,0.0,0x1d,1,13,0x00,0,0,0,0,0,0.0,00:18:de:1a:c1:df,NaN,NaN,NaN,NaN,NaN,NaN,1.0,normal
575639,0.005485,0.005485,261,261,26,1,1,1,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,2437.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-58.0,1.0,0.0,0x08,0,8,0x00,0,0,0,0,0,0.0,ff:ff:ff:ff:ff:ff,ff:ff:ff:ff:ff:ff,b0:48:7a:e2:62:23,b0:48:7a:e2:62:23,b0:48:7a:e2:62:23,0.0,801.0,1.0,normal
575640,0.008112,0.008112,124,124,26,1,1,1,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,54.0,2437.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-25.0,1.0,0.0,0x28,2,8,0x02,0,0,0,0,1,44.0,00:18:de:1a:c1:df,00:18:de:1a:c1:df,28:c6:8e:86:d3:d6,00:13:33:87:62:6d,28:c6:8e:86:d3:d6,0.0,1407.0,1.0,normal
575641,0.006147,0.006147,1524,1524,26,1,1,1,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,54.0,2437.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-25.0,1.0,0.0,0x28,2,8,0x02,0,0,0,0,1,44.0,c0:18:85:94:b6:55,c0:18:85:94:b6:55,28:c6:8e:86:d3:d6,00:13:33:87:62:6d,28:c6:8e:86:d3:d6,0.0,420.0,1.0,normal


## Convert columns with string dtypes to categorial dtypes

In [ ]:
train_df.dtypes

frame.time_delta                  float64
frame.time_delta_displayed        float64
frame.len                           int64
frame.cap_len                       int64
radiotap.length                     int64
radiotap.present.tsft               int64
radiotap.present.flags              int64
radiotap.present.channel            int64
radiotap.present.dbm_antsignal      int64
radiotap.present.antenna            int64
radiotap.present.rxflags            int64
radiotap.flags.cfp                float64
radiotap.flags.preamble           float64
radiotap.flags.wep                float64
radiotap.flags.frag               float64
radiotap.flags.fcs                float64
radiotap.flags.datapad            float64
radiotap.flags.badfcs             float64
radiotap.flags.shortgi            float64
radiotap.datarate                 float64
radiotap.channel.freq             float64
radiotap.channel.type.turbo       float64
radiotap.channel.type.cck         float64
radiotap.channel.type.ofdm        

In [ ]:
def str_to_categorical(df):
    for col in df.columns:
        if is_string_dtype(df[col]):
            df[col] = df[col].astype('category').cat.as_ordered()

In [ ]:
str_to_categorical(train_df)

In [ ]:
train_df.dtypes

frame.time_delta                   float64
frame.time_delta_displayed         float64
frame.len                            int64
frame.cap_len                        int64
radiotap.length                      int64
radiotap.present.tsft                int64
radiotap.present.flags               int64
radiotap.present.channel             int64
radiotap.present.dbm_antsignal       int64
radiotap.present.antenna             int64
radiotap.present.rxflags             int64
radiotap.flags.cfp                 float64
radiotap.flags.preamble            float64
radiotap.flags.wep                 float64
radiotap.flags.frag                float64
radiotap.flags.fcs                 float64
radiotap.flags.datapad             float64
radiotap.flags.badfcs              float64
radiotap.flags.shortgi             float64
radiotap.datarate                  float64
radiotap.channel.freq              float64
radiotap.channel.type.turbo        float64
radiotap.channel.type.cck          float64
radiotap.ch

In [ ]:
str_to_categorical(test_df)

In [ ]:
test_df.dtypes

frame.time_delta                   float64
frame.time_delta_displayed         float64
frame.len                            int64
frame.cap_len                        int64
radiotap.length                      int64
radiotap.present.tsft                int64
radiotap.present.flags               int64
radiotap.present.channel             int64
radiotap.present.dbm_antsignal       int64
radiotap.present.antenna             int64
radiotap.present.rxflags             int64
radiotap.flags.cfp                 float64
radiotap.flags.preamble            float64
radiotap.flags.wep                 float64
radiotap.flags.frag                float64
radiotap.flags.fcs                 float64
radiotap.flags.datapad             float64
radiotap.flags.badfcs              float64
radiotap.flags.shortgi             float64
radiotap.datarate                  float64
radiotap.channel.freq              float64
radiotap.channel.type.turbo        float64
radiotap.channel.type.cck          float64
radiotap.ch

## Drop categorical columns which have so many categories

In [ ]:
category_columns = []
for col in train_df.columns:
    if is_categorical_dtype(train_df[col]):
        category_columns.append(col)

In [ ]:
category_columns

['wlan.fc.type_subtype',
 'wlan.fc.ds',
 'wlan.ra',
 'wlan.da',
 'wlan.ta',
 'wlan.sa',
 'wlan.bssid',
 'class']

In [ ]:
def drop_categorical_columns(df, max_categories=10):
    drop_columns = []
    for col in df.columns:
        if is_categorical_dtype(df[col]) and len(df[col].cat.categories)>max_categories:
            drop_columns.append(col)
    df.drop(columns=drop_columns, inplace=True)

In [ ]:
drop_categorical_columns(train_df)
drop_categorical_columns(test_df)

In [ ]:
print("Training set shape after dropping categorical columns which have so many categories: ", train_df.shape)
print("Test set shape after dropping categorical columns which have so many categoriest: ", test_df.shape)

Training set shape after dropping categorical columns which have so many categories:  (1795575, 49)
Test set shape after dropping categorical columns which have so many categoriest:  (575643, 49)


## Convert categorical columns to numeric

In [ ]:
def category_to_int(df):
    for col in df.columns:
        if not is_numeric_dtype(df[col]):
            df[col] = pd.Categorical(df[col]).codes + 1

In [ ]:
category_to_int(train_df)
category_to_int(test_df)

In [ ]:
train_df.dtypes

frame.time_delta                  float64
frame.time_delta_displayed        float64
frame.len                           int64
frame.cap_len                       int64
radiotap.length                     int64
radiotap.present.tsft               int64
radiotap.present.flags              int64
radiotap.present.channel            int64
radiotap.present.dbm_antsignal      int64
radiotap.present.antenna            int64
radiotap.present.rxflags            int64
radiotap.flags.cfp                float64
radiotap.flags.preamble           float64
radiotap.flags.wep                float64
radiotap.flags.frag               float64
radiotap.flags.fcs                float64
radiotap.flags.datapad            float64
radiotap.flags.badfcs             float64
radiotap.flags.shortgi            float64
radiotap.datarate                 float64
radiotap.channel.freq             float64
radiotap.channel.type.turbo       float64
radiotap.channel.type.cck         float64
radiotap.channel.type.ofdm        

## Handle missing value

In [ ]:
def fix_missing(df):
    for col in df.columns:
        if df[col].isna().sum():
            df[col] = df[col].fillna(df[col].median())

In [ ]:
fix_missing(train_df)
fix_missing(test_df)

## Standard normalization

In [ ]:
def standard_nor(df):
    for col in df.columns:
        df[col] = (df[col] - np.mean(df[col])) / np.std(df[col])

In [ ]:
train_df.iloc[:, :-1]

Index(['frame.time_delta', 'frame.time_delta_displayed', 'frame.len',
       'frame.cap_len', 'radiotap.length', 'radiotap.present.tsft',
       'radiotap.present.flags', 'radiotap.present.channel',
       'radiotap.present.dbm_antsignal', 'radiotap.present.antenna',
       'radiotap.present.rxflags', 'radiotap.flags.cfp',
       'radiotap.flags.preamble', 'radiotap.flags.wep', 'radiotap.flags.frag',
       'radiotap.flags.fcs', 'radiotap.flags.datapad', 'radiotap.flags.badfcs',
       'radiotap.flags.shortgi', 'radiotap.datarate', 'radiotap.channel.freq',
       'radiotap.channel.type.turbo', 'radiotap.channel.type.cck',
       'radiotap.channel.type.ofdm', 'radiotap.channel.type.2ghz',
       'radiotap.channel.type.5ghz', 'radiotap.channel.type.passive',
       'radiotap.channel.type.dynamic', 'radiotap.channel.type.gfsk',
       'radiotap.channel.type.gsm', 'radiotap.channel.type.sturbo',
       'radiotap.channel.type.half', 'radiotap.channel.type.quarter',
       'radiotap.dbm_ants